In [5]:
import pandas as pd
import numpy as np
import random

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer

from keras import optimizers
from keras.models import load_model
from keras.models import Model, Input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import multi_gpu_model, to_categorical
from keras.layers import Dense, Embedding, LSTM, GRU, Dropout, Bidirectional


### Model

In [2]:
max_len   = 200
max_vocab = 2000
embed_dim = 300
hid_dim   = 500
hid_dim2  = 300
hid_dim3  = 100
lstm_out  = 50

# Input
deep_input = Input(shape=(max_len, ), name = 'deep_input')
deep_embed = Embedding(max_vocab, embed_dim, input_length = max_len)(deep_input)


# Hidden layer
deep_lstm = Bidirectional(LSTM(hid_dim, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True),name = 'bilstm_1')(deep_embed)
deep_lstm = Bidirectional(LSTM(hid_dim2, dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True), name = 'bilstm_2')(deep_lstm)
deep_lstm = Bidirectional(LSTM(hid_dim3, dropout = 0.2, recurrent_dropout = 0.2), name = 'bilstm_3')(deep_lstm)

# Outputs
deep_output = Dense(lstm_out, activation = 'relu')(deep_lstm)
deep_output = Dense(3, activation = 'softmax', name = 'deep_output')(deep_output)
deep_model  = Model(inputs = deep_input, outputs = deep_output)

# Define loss function and optimizer
deep_model.compile(loss = 'categorical_crossentropy', 
                   optimizer = optimizers.Adam(lr = 1e-3), 
                   metrics = ['accuracy'])

deep_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
deep_input (InputLayer)      (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 300)          600000    
_________________________________________________________________
bilstm_1 (Bidirectional)     (None, 200, 1000)         3204000   
_________________________________________________________________
bilstm_2 (Bidirectional)     (None, 200, 600)          3122400   
_________________________________________________________________
bilstm_3 (Bidirectional)     (None, 200)               560800    
_________________________________________________________________
dense_1 (Dense)              (None, 50)                10050     
_________________________________________________________________
deep_output (Dense)          (None, 3)                 153       
Total para

### Reload model

In [32]:
# Create a function for sentiment prediction
def predict_label(model_file, input_texts, max_vocab = 2000, max_len = 200):
    
    # Reload model
    model = load_model(model_file)
    print(model.summary())
    
    # Preprocess input texts
    tokenizer = Tokenizer(num_words = max_vocab, split = ' ', char_level = False)
    tokenizer.fit_on_texts(input_texts.values)
    X = tokenizer.texts_to_sequences(input_texts.values)
    X = pad_sequences(X, maxlen = max_len, padding = 'post', truncating = 'post')

    # Predict label
    output_label = model.predict(X)
    output_label = np.argmax(output_label, axis = 1)
    
    print("Predited labels: ", output_label)
    
    return(output_label)

In [34]:
all_Data = pd.read_pickle('all_Data.pkl')
pred_labels = predict_label(model_file = 'blstm3_w_dp_gpu_200_100_1024_fold_3.h5', input_texts = all_Data['Texts'][:5])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_1_input (InputLayer)  (None, 200)          0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 200)          0           embedding_1_input[0][0]          
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 200)          0           embedding_1_input[0][0]          
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 200)          0           embedding_1_input[0][0]          
__________________________________________________________________________________________________
lambda_4 (